In [3]:
import pandas as pd
import re
from collections import defaultdict

In [12]:
df_teeltplan = pd.read_excel('data/Beemstergaard Teeltplan 2023.xlsx')
df_overzicht = pd.read_excel('data/Beemstergaard Overzicht 2023.xlsx', header=1)

/Users/svenvanderburg/miniforge3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [13]:
df_teeltplan.head()

,,Gewas,Ras,Plantenfamilie,Gewasgroep\nVelt,Gewasgroep \nBeemstergaard,Aantal bedden,Bed\nnummer,DZ / TP / JONG,Combinatieteelt,...,Dagen in opkweek,Zaden per gram,Zaden nodig per plant,Zaden nodig in grammen,Prijs per gram,Leverancier,Zaadkosten,Opmerkingen zaadbestelling,Zaad aanwezig (g),Opmerkingen
0,NaN,Komkommer,Arola,Komkommerachtig,Vrucht,Cucurbits,Kas,Kas,TP,NaN,...,NaN,40.0,1.2,0.900000,NaN,Bingenheim G176,NaN,NaN,,NaN
1,NaN,Komkommer,Zilveren snijkomkommer,Komkommerachtig,Vrucht,Cucurbits,Kas,Kas,TP,NaN,...,NaN,40.0,1.2,0.900000,NaN,Kokopelli,NaN,NaN,NaN,NaN
2,NaN,Komkommer,Luffa sponskomkommer,Komkommerachtig,Vrucht,Cucurbits,Kas,Kas,TP,NaN,...,NaN,40.0,1.2,0.150000,NaN,Kokopelli,NaN,NaN,NaN,NaN
3,NaN,Augurk,Liefje,Komkommerachtig,Vrucht,Cucurbits,Kas,Kas,TP,NaN,...,NaN,40.0,1.2,0.300000,NaN,Bingenheim G776,NaN,NaN,NaN,NaN
4,NaN,Aubergine,Rosa Bianca,Nachtschade,Vrucht,Cucurbits,Kas,Kas,TP,NaN,...,NaN,230.0,1.2,0.078261,NaN,Kokopelli,NaN,NaN,NaN,NaN


In [14]:
df_teeltplan.columns

Index([' ', 'Gewas', 'Ras', 'Plantenfamilie', 'Gewasgroep\nVelt',
       'Gewasgroep \nBeemstergaard', 'Aantal bedden', 'Bed\nnummer',
       'DZ / TP / JONG', 'Combinatieteelt', 'Afstand tussen rij',
       'Afstand in rij', 'Rijen per bed', 'Planten nodig', 'Zaaien (week)',
       'Zaaidatum', 'Planten (week)', 'Plantdatum', 'Verwachte oogst (week)',
       'Startdatum oogst', 'Eindweek\noogst', 'Einddatum oogst',
       'Dagen tot volgroeiing', 'Dagen in tuin', 'Dagen in opkweek',
       'Zaden per gram', 'Zaden nodig per plant', 'Zaden nodig in grammen',
       'Prijs per gram', 'Leverancier', 'Zaadkosten',
       'Opmerkingen zaadbestelling', 'Zaad aanwezig (g)', 'Opmerkingen'],
      dtype='object')

In [15]:
df_overzicht.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Weeknummer,1,2,3,4,5,6,...,43,44,45,46,47,48,49,50,51,52
0,NaN,Gewasgroep,NaN,Maand (2023),Jan,NaN,NaN,NaN,Feb,NaN,...,NaN,Nov,NaN,NaN,NaN,Dec,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,A1.01,Rabarber,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Combinatie,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_teeltplan.loc[df_teeltplan['Gewas'] == 'Pompoen']

,,Gewas,Ras,Plantenfamilie,Gewasgroep\nVelt,Gewasgroep \nBeemstergaard,Aantal bedden,Bed\nnummer,DZ / TP / JONG,Combinatieteelt,...,Dagen in opkweek,Zaden per gram,Zaden nodig per plant,Zaden nodig in grammen,Prijs per gram,Leverancier,Zaadkosten,Opmerkingen zaadbestelling,Zaad aanwezig (g),Opmerkingen
36,NaN,Pompoen,Butternut Rugosa,Komkommerachtig,Vrucht,Cucurbits,3,"B1.01, B1.02, B2.02",TP,NaN,...,NaN,8.0,1.2,3.375,NaN,Zelf,NaN,NaN,NaN,NaN
37,NaN,Pompoen,Marina Do Chioggia,Komkommerachtig,Vrucht,Cucurbits,2,"B3.01, B3.02",TP,NaN,...,NaN,8.0,1.2,2.250,NaN,Zelf,NaN,NaN,NaN,NaN
38,NaN,Pompoen,Musqueee De Provence,Komkommerachtig,Vrucht,Cucurbits,3,"A2.01, A2.02, A3.01",TP,NaN,...,NaN,8.0,1.2,3.375,NaN,Zelf,NaN,NaN,15 g,NaN
39,NaN,Pompoen,Groene Hokkaido / Kabosha,Komkommerachtig,Vrucht,Cucurbits,3,"A3.02, A4.01, A4.02",TP,NaN,...,NaN,8.0,1.2,3.375,NaN,Zelf,NaN,NaN,1 g,NaN
40,NaN,Pompoen,Spaghetti,Komkommerachtig,Vrucht,Cucurbits,2,"A5.01, A5.02",TP,NaN,...,NaN,8.0,1.2,2.250,NaN,Zelf,NaN,NaN,genoeg aanwezig,NaN
41,NaN,Pompoen,Uchiki kuri,Komkommerachtig,Vrucht,Cucurbits,2,"A6.01, A6.02",TP,Inkarnaatklaver,...,NaN,8.0,1.2,2.250,NaN,Zelf,NaN,NaN,NaN,Combinatieteelt is experiment


In [18]:
data = defaultdict(set)
def match_to_teeltplan(vegetable, weeknumber, bednumber):
    matches_vegetable = df_teeltplan['Gewas'] == vegetable
    vegetable_rows = df_teeltplan[matches_vegetable]
    matches_zaaien = vegetable_rows['Zaaien (week)'] == weeknumber
    matches_planten = vegetable_rows['Planten (week)'] == weeknumber
    if len(vegetable_rows) and (any(matches_zaaien) or any(matches_planten)):
        data[vegetable].add(bednumber)
    else:
        print(f'No match for {vegetable} in {weeknumber}')

bed_number = None
for i, row in df_overzicht.iterrows():
    if re.match(r'[A-Z]\d\.\d\d', str(row['Weeknummer'])):
        bed_number = str(row['Weeknummer'])
    the_year = row.iloc[4:]
    vegetables = the_year[~the_year.isnull()]
    for weeknumber, vegetable in vegetables.iteritems():
        match_to_teeltplan(vegetable, weeknumber, bed_number)


No match for Jan in 1
No match for Feb in 5
No match for Mrt in 9
No match for Apr in 14
No match for Mei in 18
No match for Jun in 22
No match for Jul in 27
No match for Aug in 31
No match for Sep in 35
No match for Okt in 40
No match for Nov in 44
No match for Dec in 48
No match for Rabarber in 1
No match for Rabarber in 1
No match for Rabarber in 1
No match for Winterpostelein in 1
No match for Groenbemester in 41
No match for Groenbemester in 41
No match for Biet (normaal) in 16
No match for Spinazie (winter)  in 34
No match for Groenbemester in 37
No match for Ui (7m) in 1
No match for Groenbemester in 44
No match for Broccoli (herfst) in 22
No match for Ui in 1
No match for Groenbemester in 42
No match for Kapucijner (Blauwschokker) in 9
No match for Nateelt in 30
No match for Groenbemester in 31
No match for Knoflook in 1
No match for Wortel (zomer), experiment in 17
No match for Groenbemester in 36
No match for Groenbemester in 35
No match for Pastinaak (vroeg) in 8
No match fo

/var/folders/ff/lzvrl_f53bl3fwlbvnsw4f840000gn/T/ipykernel_24641/962880471.py:18: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for weeknumber, vegetable in vegetables.iteritems():


In [93]:
data

defaultdict(set,
            {'Pompoen': {'A2.01',
              'A2.02',
              'A3.01',
              'A3.02',
              'A4.01',
              'A4.02',
              'A5.01',
              'A5.02',
              'A6.01',
              'A6.02'},
             'Broccoli (herfst)': {'A2.05'},
             'Paksoi (herfst)': {'A2.06'},
             'Aardappel (vroeg)': {'A2.09',
              'A2.10',
              'A4.09',
              'A4.10',
              'A5.10'},
             'Winterpostelein': {'A2.09'},
             'Palmkool (herfst)': {'A3.05'},
             'Radijs (lente)': {'A3.06'},
             'Radijs (herfst)': {'A3.06'},
             'Koolrabi (vroeg)': {'A3.06'},
             'Savooiekool (vroeg)': {'A4.06'},
             'Tuinboon': {'A4.07', 'A4.08'},
             'Bloemkool (vroeg vrijsters)': {'A5.05'},
             'Bloemkool (weeuwen)': {'A5.05'},
             'Bloemkool (herfst)': {'A5.06'},
             'Doperwt': {'A5.07'},
             'Boerenkool

In [5]:
data = list()
mapper = {
    'zaaien': 'Zaaien (week)',
    'planten': 'Planten (week)',
    'verwachte oogst': 'Verwachte oogst (week)'} 
for week in range(1, 53):
    for activity in mapper.keys():
        selection = df_teeltplan[df_teeltplan[mapper[activity]] == week]
        for gewas in set(selection['Gewas']): 
            data.append({'Week': week, 'Wat doen?': f'{gewas} {activity}'})
in_time_df = pd.DataFrame(data)

In [6]:
in_time_df

,Week,Wat doen?
0,3,Peul (vroeg) zaaien
1,3,Aubergine zaaien
2,3,Peper zaaien
3,3,Paprika zaaien
4,5,Rodekool (vroeg) zaaien
...,...,...
163,40,Pompoen verwachte oogst
164,41,Spruitkool (middelvroeg) verwachte oogst
165,42,Winterpostelein verwachte oogst
166,44,Aardpeer verwachte oogst


In [7]:
in_time_df.to_excel('data/planning-per-week.xlsx', index=False)